# Kaggle Project

https://inclass.kaggle.com/c/how-many-bikes/data

season - season of the record (integer, 1-4)
1: Spring
2: Summer
3: Fall
4: Winter

yr - year of the record(integer, 0-1)
0: 2011
1: 2012

wheathersit - weather situation (integer, 1-4)
1: clear, few clouds, or partly cloudy
2: mist (no precipitation)
3: light rain or light snow
4: heavy rain, hail, or snow


* Load the data
* How many features do you have? How are they encoded?
* Plot histograms to visualize the distribution of your features. Should you scale or normalize them?
* Set up a cross-validation that you will use for all your evaluations. Notice there is a 'random_state' parameter to the cross-validation methods of scikit-learn, that you can use to ensure you always get the same splits. 
* [Optional] To go one step further in ensuring a fair comparison of your algorithms, you can use multiple repeats of the cross-validation procedure (using different splits each time), and report the mean & standard deviation over the repeats of the performance obtained. If you do this, you can report standard deviations in plots by using error bars.
* Evaluate the performance of a linear regression on your data. Which evaluation metric are you using? See http://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics for help defining one.
* Submit a linear regression predictor to the leaderboard.

In [64]:
import numpy as np
from __future__ import print_function
from __future__ import division
import os
import subprocess

from time import time
from operator import itemgetter
import math

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
from sklearn.cross_validation import  cross_val_score

from sklearn import linear_model
from sklearn import preprocessing
from sklearn import metrics
from sklearn import grid_search
from sklearn.metrics import mean_squared_error
from sklearn import cross_validation
from sklearn import neighbors
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn import ensemble
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier  
from sklearn.metrics import r2_score

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Load the data
X = np.loadtxt('Kaggle/train.csv', delimiter=',', skiprows=1, usecols=range(2, 13))
y = np.loadtxt('Kaggle/train.csv', delimiter=',', skiprows=1, usecols=[14])
z = np.loadtxt('Kaggle/test.csv', delimiter=',', skiprows=1, usecols=range(2, 13))

In [3]:
# Load the data
XX = np.loadtxt('Kaggle/tra.csv', delimiter=',', skiprows=1, usecols=range(2, 18))
yy = np.loadtxt('Kaggle/tra.csv', delimiter=',', skiprows=1, usecols=[19])
zz = np.loadtxt('Kaggle/tes.csv', delimiter=',', skiprows=1, usecols=range(2, 18))

In [4]:
# Set up a stratified 10-fold cross-validation

#folds = cross_validation.StratifiedKFold(y, 200, shuffle=True)
#folds = cross_validation.train_test_split(y, test_size = 0.1, random_state=None)
#folds = cross_validation.KFold(len(y), 10, shuffle=True)

kf = cross_validation.KFold(len(y), n_folds=5, shuffle=True, random_state=4)

"""
for train, test in kf_total:
    print train, '\n', test, '\n\n'
    print train.shape
    print test.shape
"""
# http://scikit-learn.org/0.17/modules/generated/sklearn.cross_validation.train_test_split.html#sklearn.cross_validation.train_test_split
# Split arrays or matrices into random train and test subsets

"\nfor train, test in kf_total:\n    print train, '\n', test, '\n\n'\n    print train.shape\n    print test.shape\n"

https://randomforests.wordpress.com/2014/02/02/basics-of-k-fold-cross-validation-and-gridsearchcv-in-scikit-learn/

get different combinations of train/test data, essentially giving you ‘more’ data for validation from your original data. The number of times you ‘switch around’ the train/test data is the number of folds. Therefore, 10-Fold Cross Validation will yield 10 sets of train/test data, 5-Fold Cross Validation will yield 5 sets, and so forth.

**random_state** controls the degree of randomness.


## cross-validation functions

`split the data into 10 parts
fit on 9-parts
test accuracy on the remaining part`

This is repeated on all combinations to produce ten estimates of the accuracy of the model using the current parameter setting. Typically the mean and standard deviation of the ten scores is reported. So, if we use the setting from the previous post, we get:

http://chrisstrelioff.ws/sandbox/2015/06/25/decision_trees_in_python_again_cross_validation.html

In [5]:
def report(grid_scores, n_top=3):
    """Report top n_top parameters settings, default n_top=3.

    Args
    ----
    grid_scores -- output from grid or random search
    n_top -- how many to report, of top models

    Returns
    -------
    top_params -- [dict] top parameter settings found in
                  search
    """
    top_scores = sorted(grid_scores,
                        key=itemgetter(1),
                        reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print(("Mean validation score: "
               "{0:.3f} (std: {1:.3f})").format(
               score.mean_validation_score,
               np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

    return top_scores[0].parameters

def run_gridsearch(X, y, clf, param_grid, cv=5):
    """Run a grid search for best Decision Tree parameters.

    Args
    ----
    X -- features
    y -- targets (classes)
    cf -- scikit-learn Decision Tree
    param_grid -- [dict] parameter settings to test
    cv -- fold of cross-validation, default 5

    Returns
    -------
    top_params -- [dict] from report()
    """
    clf.n_jobs = -1    
    grid_search = GridSearchCV(clf,
                               param_grid=param_grid,
                               cv=cv)
    start = time()
    grid_search.fit(X, y)

    print(("\nGridSearchCV took {:.2f} "
           "seconds for {:d} candidate "
           "parameter settings.").format(time() - start,
                len(grid_search.grid_scores_)))

    top_params = report(grid_search.grid_scores_, 3)
    return  top_params


def run_randomsearch(X, y, clf, para_dist, cv=5,
                     n_iter_search=20):
    """Run a random search for best Decision Tree parameters.

    Args
    ----
    X -- features
    y -- targets (classes)
    cf -- scikit-learn Decision Tree
    param_dist -- [dict] list, distributions of parameters
                  to sample
    cv -- fold of cross-validation, default 5
    n_iter_search -- number of random parameter sets to try,
                     default 20.

    Returns
    -------
    top_params -- [dict] from report()
    """
    clf.n_jobs = -1    
    random_search = RandomizedSearchCV(clf,
                        param_distributions=param_dist,
                        n_iter=n_iter_search)

    start = time()
    random_search.fit(X, y)
    print(("\nRandomizedSearchCV took {:.2f} seconds "
           "for {:d} candidates parameter "
           "settings.").format((time() - start),
                               n_iter_search))

    top_params = report(random_search.grid_scores_, 3)
    return  top_params

# Linear Regression 

#### Cross validation without scaling nor normalization

In [6]:
def line_CV_fresh(design_matrix, labels, classifier, cv_folds):
    pred = np.zeros(labels.shape)
    RMSE_ave = np.zeros((len(cv_folds),1))
    
    for ix, (tr, te) in enumerate(cv_folds):
        Xtr = design_matrix[tr, :]
        ytr = labels[tr]
        Xte = design_matrix[te, :]
        
        # Scale data
        scaler = preprocessing.StandardScaler() # create scaler
        Xtr = scaler.fit_transform(Xtr) # fit the scaler to the training data 
                                    # and transform training data
        Xte = scaler.transform(Xte) # transform test data
        
        classifier.fit(Xtr, ytr)
        yte_pred = classifier.predict(Xte)
        pred[te] = yte_pred[:]
        
        assert len(pred[te]) == len(labels[te])
        RMSE = np.sqrt(np.mean(np.power(np.log1p(pred[te])-np.log1p(labels[te]), 2)))
        print("Root Mean Squared Logarithmic Error (higher; worse): %0.3f" % RMSE)
        RMSE_ave[ix] = RMSE
    
    print ("RMSLE Mean: %0.3f" % np.mean(RMSE_ave))
    print ("\n")
    return pred


def line_CV_fresh_para(design_matrix, labels, classifier, cv_folds):
    pred = np.zeros(labels.shape)
    RMSE_ave = np.zeros((len(cv_folds),1))
    
    for ix, (tr, te) in enumerate(cv_folds):
        Xtr = design_matrix[tr, :]
        ytr = labels[tr]
        Xte = design_matrix[te, :]
        
        # Scale data
        scaler = preprocessing.StandardScaler() # create scaler
        Xtr = scaler.fit_transform(Xtr) # fit the scaler to the training data 
                                    # and transform training data
        Xte = scaler.transform(Xte) # transform test data
        
        classifier.fit(Xtr, ytr)
        yte_pred = classifier.predict(Xte)
        pred[te] = yte_pred[:]
        
        print ("Best parameters for fold ",ix,"\n")
        print (classifier.best_params_)
        assert len(pred[te]) == len(labels[te])
        RMSE = np.sqrt(np.mean(np.power(np.log1p(pred[te])-np.log1p(labels[te]), 2)))
        print ("Root Mean Squared Logarithmic Error (higher-> worse): %0.3f" % RMSE)
        RMSE_ave[ix] = RMSE
    
    print ("RMSLE Mean: %0.3f" % np.mean(RMSE_ave))
    print ("\n")
    return pred

### cross validation + scaling

In [7]:
def line_CV_spicy(design_matrix, labels, classifier, cv_folds):
    pred = np.zeros(labels.shape)
    RMSE_ave = np.zeros((len(cv_folds),1))

    for ix, (tr, te) in enumerate(cv_folds):
        Xtr = design_matrix[tr, :]
        ytr = labels[tr]
        Xte = design_matrix[te, :]
        
        # Scale data
        scaler = preprocessing.StandardScaler() # create scaler
        Xtr = scaler.fit_transform(Xtr) # fit the scaler to the training data 
                                    # and transform training data
        Xte = scaler.transform(Xte) # transform test data
                
        classifier.n_jobs = -1    
        classifier.fit(Xtr, ytr)
        yte_pred = classifier.predict(Xte)
        pred[te] = yte_pred[:]
        
        assert len(pred[te]) == len(labels[te])
        RMSE = np.sqrt(np.mean(np.power(np.log1p(pred[te])-np.log1p(labels[te]), 2)))
        print ("Root Mean Squared Logarithmic Error (higher; worse): %0.3f" % RMSE)
        RMSE_ave[ix] = RMSE
    
    print ("RMSLE Mean: %0.3f" % np.mean(RMSE_ave))
    print ("\n")

    return pred

def line_CV_spicy_para(design_matrix, labels, classifier, cv_folds):
    pred = np.zeros(labels.shape)
    RMSE_ave = np.zeros((len(cv_folds),1))

    for ix, (tr, te) in enumerate(cv_folds):
        Xtr = design_matrix[tr, :]
        ytr = labels[tr]
        Xte = design_matrix[te, :]
        
        # Scale data
        scaler = preprocessing.StandardScaler() # create scaler
        Xtr = scaler.fit_transform(Xtr) # fit the scaler to the training data 
                                    # and transform training data
        Xte = scaler.transform(Xte) # transform test data
                
        classifier.n_jobs = -1    
        classifier.fit(Xtr, ytr)
        yte_pred = classifier.predict(Xte)
        pred[te] = yte_pred[:]
        
        print ("Best parameters for fold ",ix,"\n")
        print (classifier.best_params_)
        assert len(pred[te]) == len(labels[te])
        RMSE = np.sqrt(np.mean(np.power(np.log1p(pred[te])-np.log1p(labels[te]), 2)))
        print ("Root Mean Squared Logarithmic Error (higher; worse): %0.3f" % RMSE)
        RMSE_ave[ix] = RMSE
    
    print ("RMSLE Mean: %0.3f" % np.mean(RMSE_ave))
    print ("\n")
    return pred

### cross validation + scaling + normalization 

In [8]:
def line_CV_very_spicy(design_matrix, labels, classifier, cv_folds):
    pred = np.zeros(labels.shape)
    RMSE_ave = np.zeros((len(cv_folds),1))
    X_norm = preprocessing.normalize(design_matrix)
  
    for ix, (tr, te) in enumerate(cv_folds):
        Xtr = X_norm[tr, :]
        ytr = labels[tr]
        Xte = X_norm[te, :]
        
        # Scale data
        scaler = preprocessing.StandardScaler()
        Xtr = scaler.fit_transform(Xtr)
        Xte = scaler.transform(Xte)

        classifier.n_jobs = -1
        classifier.fit(Xtr, ytr)
        yte_pred = classifier.predict(Xte)
        pred[te] = yte_pred[:]
        
        assert len(pred[te]) == len(labels[te])
        RMSE = np.sqrt(np.mean(np.power(np.log1p(pred[te])-np.log1p(labels[te]), 2)))
        print ("Root Mean Squared Logarithmic Error (higher; worse): %0.3f" % RMSE)
        RMSE_ave[ix] = RMSE
    
    print ("RMSLE Mean: %0.3f" % np.mean(RMSE_ave))
    print ("\n")

    return pred

def line_CV_very_spicy_para(design_matrix, labels, classifier, cv_folds):
    pred = np.zeros(labels.shape)
    RMSE_ave = np.zeros((len(cv_folds),1))
    X_norm = preprocessing.normalize(design_matrix)
    
    for ix, (tr, te) in enumerate(cv_folds):
        Xtr = X_norm[tr, :]
        ytr = labels[tr]
        Xte = X_norm[te, :]
        
        # Scale data
        scaler = preprocessing.StandardScaler()
        Xtr = scaler.fit_transform(Xtr)
        Xte = scaler.transform(Xte)

        classifier.n_jobs = -1
        classifier.fit(Xtr, ytr)
        pred[te] = classifier.predict(Xte)
        
        print ("Best parameters for fold ",ix,"\n")
        print (classifier.best_params_)
        assert len(pred[te]) == len(labels[te])
        RMSE = np.sqrt(np.mean(np.power(np.log1p(pred[te])-np.log1p(labels[te]), 2)))
        print ("Root Mean Squared Logarithmic Error (higher; worse): %0.3f" % RMSE)
        RMSE_ave[ix] = RMSE
    
    print ("RMSLE Mean: %0.3f" % np.mean(RMSE_ave))
    print ("\n")
    return pred

https://inclass.kaggle.com/c/how-many-bikes/details/evaluation

#### Kaggle Evaluation Method: Root Mean squared logarithmic error
Submissions are evaluated one the Root Mean Squared Logarithmic Error (RMSLE). This metric is suitable when predicting values across a large range of orders of magnitudes. It avoids penalizing large differences in prediction when both the predicted and the true number are large: predicting 5 when the true value is 50 is penalized more than predicting 500 when the true value is 545.

Code-Reference from:
https://ajourneyintodatascience.quora.com/Custom-evaluation-function-and-early-stopping-for-xgboost-with-k-fold-validation-Python

### Cross Validation evaluated by mean squared error (MSE) & R Squared metric 

In [72]:
def CV_with_MSE(design_matrix, labels, classifier, cv_folds):
    pred = np.zeros(labels.shape)
    RMSE_ave = np.zeros((len(cv_folds),1))
    RS_ave = np.zeros((len(cv_folds),1))
    MAPE_ave = np.zeros((len(cv_folds),1))
    MASE_ave = np.zeros((len(cv_folds),1))
    X_norm = preprocessing.normalize(design_matrix)
    
    for ix, (tr, te) in enumerate(cv_folds):
        Xtr = X_norm[tr, :]
        ytr = labels[tr]
        Xte = X_norm[te, :]
        
        # Scale data
        scaler = preprocessing.StandardScaler()
        Xtr = scaler.fit_transform(Xtr)
        Xte = scaler.transform(Xte)
        
        classifier.n_jobs = -1
        classifier.fit(Xtr, ytr)
        yte_pred = classifier.predict(Xte)
        pred[te] = yte_pred[:]
        
        RMSE = mean_squared_error(labels[te], pred[te]) ** 0.5
        RS = r2_score(labels[te], pred[te])
        MAPE = np.mean(np.abs((labels[te] - pred[te]) / labels[te])) * 100
        num = Xte.shape[0]
        diffe = np.abs(  np.diff(Xte) ).sum()/(num-1)
        errors = np.abs(pred[te] - labels[te])
        MASE = errors.mean() / diffe
        
        print ("Root Mean Squared Error: %0.3f, R Square mean: %0.3f" % (RMSE, RS))
        print ("Mean absolute percentage error: %0.3f, Mean absolute scaled error: %0.3f" % (MAPE, MASE))
        RMSE_ave[ix] = RMSE
        RS_ave[ix] = RS
        MAPE_ave[ix] = MAPE
        MASE_ave[ix] = MASE
    
    print ("RMSE Mean: %0.3f, R Square mean: %0.3f" % (np.mean(RMSE_ave), np.mean(RS_ave)))
    print ("MAPE: %0.3f, MASE: %0.3f" % (np.mean(MAPE_ave), np.mean(MASE_ave)))
    print ("\n")

    return pred

def CV_with_MSE_para(design_matrix, labels, classifier, cv_folds):
    pred = np.zeros(labels.shape)
    MSE_ave = np.zeros((len(cv_folds),1))
    RS_ave = np.zeros((len(cv_folds),1))
    MAPE_ave = np.zeros((len(cv_folds),1))
    MASE_ave = np.zeros((len(cv_folds),1))
    X_norm = preprocessing.normalize(design_matrix)
    
    for ix, (tr, te) in enumerate(cv_folds):
        Xtr = X_norm[tr, :]
        ytr = labels[tr]
        Xte = X_norm[te, :]
        
        # Scale data
        scaler = preprocessing.StandardScaler()
        Xtr = scaler.fit_transform(Xtr)
        Xte = scaler.transform(Xte)
           
        classifier.n_jobs = -1
        classifier.fit(Xtr, ytr)
        pred[te] = classifier.predict(Xte)
        
        print ("Best parameters for fold ",ix,"\n")
        print (classifier.best_params_)

        MSE = mean_squared_error(labels[te], pred[te])
        RS = r2_score(labels[te], pred[te])
        MAPE = np.mean(np.abs((labels[te] - pred[te]) / labels[te])) * 100
        num = Xte.shape[0]
        diffe = np.abs(  np.diff(Xte) ).sum()/(num-1)
        errors = np.abs(pred[te] - labels[te])
        MASE = errors.mean() / diffe
        
        print ("Root Mean Squared Error: %0.3f, R Square mean: %0.3f" % (RMSE, RS))
        print ("Mean absolute percentage error: %0.3f, Mean absolute scaled error: %0.3f" % (MAPE, MASE))
        RMSE_ave[ix] = RMSE
        RS_ave[ix] = RS
        MAPE_ave[ix] = MAPE
        MASE_ave[ix] = MASE
    
    print ("RMSE Mean: %0.3f, & R Square mean: %0.3f" % (np.mean(RMSE_ave), np.mean(RS_ave)))
    print ("MAPE: %0.3f, MASE: %0.3f" % (np.mean(MAPE_ave), np.mean(MASE_ave)))
    print ("\n")
    
    return pred

### Part one: Run the Data

grid search parameter dictionary

multiclass or multilabel problems -> average parameter

"macro" 
simply calculates the mean of the binary metrics, giving equal weight to each class. In problems where infrequent classes are nonetheless important, macro-averaging may be a means of highlighting their performance. On the other hand, the assumption that all classes are equally important is often untrue, such that macro-averaging will over-emphasize the typically low performance on an infrequent class.

"weighted" 
accounts for class imbalance by computing the average of binary metrics in which each class’s score is weighted by its presence in the true data sample.

"micro" 
gives each sample-class pair an equal contribution to the overall metric (except as a result of sample-weight). Rather than summing the metric per class, this sums the dividends and divisors that make up the per-class metrics to calculate an overall quotient. Micro-averaging may be preferred in multilabel settings, including multiclass classification where a majority class is to be ignored.

"samples" 
applies only to multilabel problems. It does not calculate a per-class measure, instead calculating the metric over the true and predicted classes for each sample in the evaluation data, and returning their (sample_weight-weighted) average.

Selecting average=None will return an array with the score for each class.

http://scikit-learn.org/stable/modules/model_evaluation.html#from-binary-to-multiclass-and-multilabel

In [8]:
"""print("-- Grid Parameter Search via 10-fold CV")

# set of parameters to test
param_grid = {'C': [1e1,1e2,1e3,1e4,1e5,1e6]}

LR_clf = linear_model.LogisticRegression()
LR_ts_gs = run_gridsearch(X, y, LR_clf, param_grid, cv=10)

Parameter Grid Search

"""

-- Grid Parameter Search via 10-fold CV

GridSearchCV took 384.39 seconds for 6 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.013 (std: 0.056)
Parameters: {'C': 10.0}

Model with rank: 2
Mean validation score: 0.013 (std: 0.052)
Parameters: {'C': 100.0}

Model with rank: 3
Mean validation score: 0.010 (std: 0.062)
Parameters: {'C': 1000.0}



In [12]:
clf = linear_model.LogisticRegression(C=1e2)

print ("linear regression cross validation without scaling and normalization:")
ypred_1 = line_CV_fresh(X, y, clf, kf)
print ("linear regression cross validation with scaling:")
ypred_2 = line_CV_spicy(X, y, clf, kf)
print ("linear regression cross validation with scaling + normalization:")
ypred_3 = line_CV_very_spicy(X, y, clf, kf)

linear regression cross validation without scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 1.339
Root Mean Squared Logarithmic Error (higher; worse): 1.292
Root Mean Squared Logarithmic Error (higher; worse): 1.173
Root Mean Squared Logarithmic Error (higher; worse): 1.264
Root Mean Squared Logarithmic Error (higher; worse): 1.046
Root Mean Squared Logarithmic Error (higher; worse): 1.131
Root Mean Squared Logarithmic Error (higher; worse): 1.270
Root Mean Squared Logarithmic Error (higher; worse): 1.125
Root Mean Squared Logarithmic Error (higher; worse): 1.252
Root Mean Squared Logarithmic Error (higher; worse): 1.177
RMSLE Mean: 1.207


linear regression cross validation with scaling:
Root Mean Squared Logarithmic Error (higher; worse): 1.339
Root Mean Squared Logarithmic Error (higher; worse): 1.292
Root Mean Squared Logarithmic Error (higher; worse): 1.173
Root Mean Squared Logarithmic Error (higher; worse): 1.264
Root Mean Squared Logarithmic Error

# Logistic Regression with Regularization

### C parameter 
http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression 


### Reducing computational expense :  RandomizedSearchCV
Searching many different parameters at once may be computationally infeasible
  For example, Searching 10 parameters (each range of 1000)
    (Require 10,000 trials of CV, 100,000 model fits with 10-fold CV, 100,000 predictions with 10-fold CV)

http://www.ritchieng.com/machine-learning-efficiently-search-tuning-param/

In [ ]:
print("-- Grid Parameter Search via 10-fold CV")

# set of parameters to test
param_grid = {'C': [1e-3, 1e-2, 1., 1e-1, 1e2, 1e3]}

LR_Reg_clf = linear_model.LogisticRegression(penalty='l1')
LR_Reg_ts_gs = run_gridsearch(X, y, LR_Reg_clf, param_grid, cv=10)

-- Grid Parameter Search via 10-fold CV


/Users/Thwowu/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


## 1. L1-Regularized Logistic Regression 

In [60]:
clf_ReL1 = grid_search.GridSearchCV(linear_model.LogisticRegression(penalty='l1'), param_grid_regu)

## 2. L2-regularized logistic regression

In [61]:
clf_ReL2 = grid_search.GridSearchCV(linear_model.LogisticRegression(penalty='l2'), param_grid_regu)

In [10]:
clf_ReL1_set = linear_model.LogisticRegression(penalty='l1', C=1e-2)
clf_ReL2_set = linear_model.LogisticRegression(penalty='l2', C=1e-2)

print ("Regulizedlinear L1 regression cross validation with scaling and normalization:")
ypred_1_regu = line_CV_very_spicy(X, y, clf_ReL1_set, kf)
print ("Regulized L2 linear regression cross validation with scaling and normalization:")
ypred_2_regu = line_CV_very_spicy(X, y, clf_ReL2_set, kf)


Regulizedlinear L1 regression cross validation with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 2.669
Root Mean Squared Logarithmic Error (higher; worse): 2.670
Root Mean Squared Logarithmic Error (higher; worse): 2.773
Root Mean Squared Logarithmic Error (higher; worse): 2.947
Root Mean Squared Logarithmic Error (higher; worse): 2.794
Root Mean Squared Logarithmic Error (higher; worse): 2.960
Root Mean Squared Logarithmic Error (higher; worse): 3.335
Root Mean Squared Logarithmic Error (higher; worse): 2.888
Root Mean Squared Logarithmic Error (higher; worse): 2.871
Root Mean Squared Logarithmic Error (higher; worse): 2.923
RMSLE Mean: 2.883


Regulized L2 linear regression cross validation with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 2.165
Root Mean Squared Logarithmic Error (higher; worse): 1.904
Root Mean Squared Logarithmic Error (higher; worse): 2.153
Root Mean Squared Logarithmic Error (higher; worse): 2

In [11]:
clf_ReL1_set = linear_model.LogisticRegression(penalty='l1', C=1e-1)
clf_ReL2_set = linear_model.LogisticRegression(penalty='l2', C=1e-1)

print ("Regulizedlinear L1 regression cross validation with scaling and normalization:")
ypred_1_regu = line_CV_very_spicy(X, y, clf_ReL1_set, kf)
print ("Regulized L2 linear regression cross validation with scaling and normalization:")
ypred_2_regu = line_CV_very_spicy(X, y, clf_ReL2_set, kf)

Regulizedlinear L1 regression cross validation with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 2.405
Root Mean Squared Logarithmic Error (higher; worse): 2.491
Root Mean Squared Logarithmic Error (higher; worse): 2.905
Root Mean Squared Logarithmic Error (higher; worse): 2.867
Root Mean Squared Logarithmic Error (higher; worse): 2.408
Root Mean Squared Logarithmic Error (higher; worse): 2.763
Root Mean Squared Logarithmic Error (higher; worse): 2.815
Root Mean Squared Logarithmic Error (higher; worse): 2.838
Root Mean Squared Logarithmic Error (higher; worse): 2.786
Root Mean Squared Logarithmic Error (higher; worse): 3.079
RMSLE Mean: 2.736


Regulized L2 linear regression cross validation with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 2.094
Root Mean Squared Logarithmic Error (higher; worse): 1.798
Root Mean Squared Logarithmic Error (higher; worse): 1.996
Root Mean Squared Logarithmic Error (higher; worse): 1

In [13]:
clf_ReL1_set = linear_model.LogisticRegression(penalty='l1', C=1.)
clf_ReL2_set = linear_model.LogisticRegression(penalty='l2', C=1.)

print ("Regulizedlinear L1 regression cross validation with scaling and normalization:")
ypred_1_regu = line_CV_very_spicy(X, y, clf_ReL1_set, kf)
print ("Regulized L2 linear regression cross validation with scaling and normalization:")
ypred_2_regu = line_CV_very_spicy(X, y, clf_ReL2_set, kf)

Regulizedlinear L1 regression cross validation with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 1.554
Root Mean Squared Logarithmic Error (higher; worse): 1.523
Root Mean Squared Logarithmic Error (higher; worse): 1.413
Root Mean Squared Logarithmic Error (higher; worse): 1.507
Root Mean Squared Logarithmic Error (higher; worse): 1.262
Root Mean Squared Logarithmic Error (higher; worse): 1.521
Root Mean Squared Logarithmic Error (higher; worse): 1.449
Root Mean Squared Logarithmic Error (higher; worse): 1.554
Root Mean Squared Logarithmic Error (higher; worse): 1.576
Root Mean Squared Logarithmic Error (higher; worse): 1.747
RMSLE Mean: 1.511


Regulized L2 linear regression cross validation with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 1.696
Root Mean Squared Logarithmic Error (higher; worse): 1.591
Root Mean Squared Logarithmic Error (higher; worse): 1.491
Root Mean Squared Logarithmic Error (higher; worse): 1

In [12]:
clf_ReL1_set = linear_model.LogisticRegression(penalty='l1', C=1e1)
clf_ReL2_set = linear_model.LogisticRegression(penalty='l2', C=1e1)

print ("Regulizedlinear L1 regression cross validation with scaling and normalization:")
ypred_1_regu = line_CV_very_spicy(X, y, clf_ReL1_set, kf)
print ("Regulized L2 linear regression cross validation with scaling and normalization:")
ypred_2_regu = line_CV_very_spicy(X, y, clf_ReL2_set, kf)

Regulizedlinear L1 regression cross validation with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 0.900
Root Mean Squared Logarithmic Error (higher; worse): 0.927
Root Mean Squared Logarithmic Error (higher; worse): 0.864
Root Mean Squared Logarithmic Error (higher; worse): 0.895
Root Mean Squared Logarithmic Error (higher; worse): 0.815
Root Mean Squared Logarithmic Error (higher; worse): 0.798
Root Mean Squared Logarithmic Error (higher; worse): 0.923
Root Mean Squared Logarithmic Error (higher; worse): 0.759
Root Mean Squared Logarithmic Error (higher; worse): 0.784
Root Mean Squared Logarithmic Error (higher; worse): 0.845
RMSLE Mean: 0.851


Regulized L2 linear regression cross validation with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 1.086
Root Mean Squared Logarithmic Error (higher; worse): 0.994
Root Mean Squared Logarithmic Error (higher; worse): 1.022
Root Mean Squared Logarithmic Error (higher; worse): 1

In [14]:
clf_ReL1_set = linear_model.LogisticRegression(penalty='l1', C=1e2)
clf_ReL2_set = linear_model.LogisticRegression(penalty='l2', C=1e2)

print ("Regulizedlinear L1 regression cross validation with scaling and normalization:")
ypred_1_regu = line_CV_very_spicy(X, y, clf_ReL1_set, kf)
print ("Regulized L2 linear regression cross validation with scaling and normalization:")
ypred_2_regu = line_CV_very_spicy(X, y, clf_ReL2_set, kf)

Regulizedlinear L1 regression cross validation with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 0.828
Root Mean Squared Logarithmic Error (higher; worse): 0.896
Root Mean Squared Logarithmic Error (higher; worse): 0.868
Root Mean Squared Logarithmic Error (higher; worse): 0.896
Root Mean Squared Logarithmic Error (higher; worse): 0.835
Root Mean Squared Logarithmic Error (higher; worse): 0.699
Root Mean Squared Logarithmic Error (higher; worse): 0.917
Root Mean Squared Logarithmic Error (higher; worse): 0.802
Root Mean Squared Logarithmic Error (higher; worse): 0.867
Root Mean Squared Logarithmic Error (higher; worse): 0.780
RMSLE Mean: 0.839


Regulized L2 linear regression cross validation with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 0.803
Root Mean Squared Logarithmic Error (higher; worse): 1.005
Root Mean Squared Logarithmic Error (higher; worse): 0.844
Root Mean Squared Logarithmic Error (higher; worse): 0

In [24]:
# bypasss
# export the prediction into csv file

# yte_pred.tofile('foo.csv',sep=',',format='%.0f')

# KNN with Paramatric Grid 

**Weights:**
default options --> uniform (all points in the neighborhood are weighted equally)
another option --> distance (weights closer neighbors more heavily than further neighbors)


In [ ]:
"""# paramatric grid
metrics       = ['minkowski','euclidean','manhattan'] 
weights       = ['uniform','distance'] 
numNeighbors  = np.arange(1,30, 2)
gamma_range   = np.logspace(-5, 5, 5) #10.0**np.arange(-5,4)
param_dist    = dict(metric=metrics,weights=weights,n_neighbors=numNeighbors, gamma_range=gamma_range)

# n_iter controls the number of searches
# instantiate model
# 2 new params
# n_iter --> controls number of random combinations it will try
# random_state for reproducibility 
rand = RandomizedSearchCV(neighbors.KNeighborsClassifier(),
                          param_dist, cv=10, scoring='average_precision', n_iter=10, random_state=5)

# fit
rand.fit(X, y)

# scores
rand.grid_scores_

print(rand.best_score_)
print(rand.best_params_)
print(rand.best_estimator_)"""

In [19]:
print("-- KNN Grid Parameter Search via 10-fold CV")

# set of parameters to test
metrics       = ['minkowski','euclidean','manhattan'] 
weights       = ['uniform','distance'] #10.0**np.arange(-5,4)
numNeighbors  = np.arange(1,30, 2)
param_dist    = dict(metric=metrics,weights=weights,n_neighbors=numNeighbors)       

KNN_clf = neighbors.KNeighborsClassifier()
KNN_ts_gs = run_gridsearch(X, y, KNN_clf, param_dist, cv=5)

-- KNN Grid Parameter Search via 10-fold CV

GridSearchCV took 72.96 seconds for 90 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.020 (std: 0.012)
Parameters: {'n_neighbors': 5, 'metric': 'minkowski', 'weights': 'distance'}

Model with rank: 2
Mean validation score: 0.020 (std: 0.007)
Parameters: {'n_neighbors': 23, 'metric': 'minkowski', 'weights': 'distance'}

Model with rank: 3
Mean validation score: 0.020 (std: 0.012)
Parameters: {'n_neighbors': 5, 'metric': 'euclidean', 'weights': 'distance'}



In [25]:
#rank 1:
clf_Knn_set1 = neighbors.KNeighborsClassifier(metric = 'minkowski' ,
                                             weights = 'distance',
                                             n_neighbors = 5)
#rank 2:
clf_Knn_set2 = neighbors.KNeighborsClassifier(metric = 'minkowski',
                                             weights = 'distance',
                                             n_neighbors = 23)
#rank 3:
clf_Knn_set3 = neighbors.KNeighborsClassifier(metric =  'euclidean',
                                             weights = 'distance',
                                             n_neighbors = 5)

print ("Rank 1 KNN Regression cross validation with scaling and normalization:")
ypred_1_knn = line_CV_very_spicy(X, y, clf_Knn_set1, kf)
print ("Rank 2 KNN Regression cross validation with scaling and normalization:")
ypred_2_knn = line_CV_very_spicy(X, y, clf_Knn_set2, kf)
print ("Rank 3 KNN Regression cross validation with scaling and normalization:")
ypred_3_knn = line_CV_very_spicy(X, y, clf_Knn_set3, kf)


Rank 1 KNN Regression cross validation with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 0.692
Root Mean Squared Logarithmic Error (higher; worse): 0.632
Root Mean Squared Logarithmic Error (higher; worse): 0.589
Root Mean Squared Logarithmic Error (higher; worse): 0.546
Root Mean Squared Logarithmic Error (higher; worse): 0.583
Root Mean Squared Logarithmic Error (higher; worse): 0.612
Root Mean Squared Logarithmic Error (higher; worse): 0.619
Root Mean Squared Logarithmic Error (higher; worse): 0.598
Root Mean Squared Logarithmic Error (higher; worse): 0.577
Root Mean Squared Logarithmic Error (higher; worse): 0.639
RMSLE Mean: 0.609


Rank 2 KNN Regression cross validation with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 0.617
Root Mean Squared Logarithmic Error (higher; worse): 0.701
Root Mean Squared Logarithmic Error (higher; worse): 0.655
Root Mean Squared Logarithmic Error (higher; worse): 0.614
Root Mean Sq

In [48]:
"""neigh = neighbors.KNeighborsClassifier(n_neighbors=clf_knn.best_params_)

design_matrix = preprocessing.normalize(X)
labels = y
pred = np.zeros(z.shape)

scaler = preprocessing.StandardScaler() # create scaler
Xtr = design_matrix[:]
ytr = labels[:]
Xte = preprocessing.normalize(z)

Xtr = scaler.fit_transform(Xtr) # fit the scaler to the training data and transform training data
Xte = scaler.transform(Xte) # transform test data

classifier.n_jobs = -1
classifier.fit(Xtr, ytr)
yte_pred2 = classifier.predict(Xte)
"""

In [49]:
# yte_pred2.tofile('foo10.csv',sep=',',format='%.0f')


# Random Forest

### with "gini" and "Entropy"

In [33]:
print("-- KNN Grid Parameter Search via 5-fold CV")

# set of parameters to test
n_estimators       = [5, 10, 15, 20]
criterion          = ["gini", "entropy"]
min_samples_split  = [1, 3, 5, 10, 20]
min_samples_leaf   = [1, 3, 5, 10, 20]
max_leaf_nodes     = [2, 5, 10, 20]
param_grid_knn    = dict(n_estimators=n_estimators,
                     criterion=criterion,
                     min_samples_split=min_samples_split,
                     min_samples_leaf=min_samples_leaf, 
                     max_leaf_nodes=max_leaf_nodes
                    )      

KNN_clf = ensemble.RandomForestClassifier()
KNN_ts_gs = run_gridsearch(X, y, KNN_clf, param_grid_knn, cv=5)

-- KNN Grid Parameter Search via 5-fold CV

GridSearchCV took 1313.51 seconds for 800 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.024 (std: 0.016)
Parameters: {'min_samples_split': 20, 'max_leaf_nodes': 20, 'n_estimators': 10, 'criterion': 'gini', 'min_samples_leaf': 10}

Model with rank: 2
Mean validation score: 0.024 (std: 0.018)
Parameters: {'min_samples_split': 3, 'max_leaf_nodes': 10, 'n_estimators': 20, 'criterion': 'entropy', 'min_samples_leaf': 10}

Model with rank: 3
Mean validation score: 0.023 (std: 0.030)
Parameters: {'min_samples_split': 10, 'max_leaf_nodes': 10, 'n_estimators': 15, 'criterion': 'gini', 'min_samples_leaf': 3}



In [34]:
#rank 1:
clf_RF_set1 = ensemble.RandomForestClassifier(n_estimators  =  10,
                                               criterion = 'gini',
                                               min_samples_split = 20,
                                               min_samples_leaf = 10, 
                                               max_leaf_nodes = 20)
#rank 2:
clf_RF_set2 = ensemble.RandomForestClassifier(n_estimators  =  20,
                                               criterion = 'entropy',
                                               min_samples_split = 3,
                                               min_samples_leaf = 10, 
                                               max_leaf_nodes = 10)
#rank 3:
clf_RF_set3 = ensemble.RandomForestClassifier(n_estimators  =  15,
                                               criterion = 'gini',
                                               min_samples_split = 10,
                                               min_samples_leaf = 3, 
                                               max_leaf_nodes = 10)

print ("Rank 1 KNN Regression cross validation with scaling and normalization:")
ypred_1_RF = line_CV_very_spicy(X, y, clf_RF_set1, kf)
print ("Rank 2 KNN Regression cross validation with scaling and normalization:")
ypred_2_RF = line_CV_very_spicy(X, y, clf_RF_set2, kf)
print ("Rank 3 KNN Regression cross validation with scaling and normalization:")
ypred_3_RF = line_CV_very_spicy(X, y, clf_RF_set3, kf)

Rank 1 KNN Regression cross validation with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 1.058
Root Mean Squared Logarithmic Error (higher; worse): 1.095
Root Mean Squared Logarithmic Error (higher; worse): 0.934
Root Mean Squared Logarithmic Error (higher; worse): 0.961
Root Mean Squared Logarithmic Error (higher; worse): 0.848
Root Mean Squared Logarithmic Error (higher; worse): 0.832
Root Mean Squared Logarithmic Error (higher; worse): 0.905
Root Mean Squared Logarithmic Error (higher; worse): 0.858
Root Mean Squared Logarithmic Error (higher; worse): 1.060
Root Mean Squared Logarithmic Error (higher; worse): 1.163
RMSLE Mean: 0.971


Rank 2 KNN Regression cross validation with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 0.900
Root Mean Squared Logarithmic Error (higher; worse): 1.323
Root Mean Squared Logarithmic Error (higher; worse): 1.224
Root Mean Squared Logarithmic Error (higher; worse): 1.239
Root Mean Sq

In [ ]:
"""
Reference:
http://astrohackweek.org/blog/multi-output-random-forests.html

"""

# Random Forest Regressor

Supported criteria are “mse” for the mean squared error, which is equal to variance reduction as feature selection criterion, and “mae” for the mean absolute error.
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html


The only supported criterion is “mse” for the mean squared error.
http://stackoverflow.com/questions/24803802/optimizing-randomforestregressor-for-other-metrics


In [ ]:
# tried for 5 hours in 5 CV, running out of ram and no result

print("-- Random Forest Regressor Grid Parameter Search via 5-fold CV")

# set of parameters to test
n_estimators       = [10, 20, 50, 100, 1000]
min_samples_split  = [1, 3, 5, 10, 20]
min_samples_leaf   = [1, 3, 5, 10, 20]
max_leaf_nodes     = [2, 5, 10, 20]
param_grid_RFR    = dict(n_estimators=n_estimators,
                     min_samples_split=min_samples_split,
                     min_samples_leaf=min_samples_leaf, 
                     max_leaf_nodes=max_leaf_nodes
                    )      

RFR_clf = RandomForestRegressor()
RFR_ts_gs = run_gridsearch(X, y, RFR_clf, param_grid_RFR, cv=5)

In [88]:
#random 1:
clf_RFR_set1 = ensemble.RandomForestClassifier(n_estimators  =  3000,
                                               min_samples_split = 1,
                                               min_samples_leaf = 3, 
                                               max_leaf_nodes = 1000)
#random 2:
clf_RFR_set2 = ensemble.RandomForestClassifier(n_estimators  =  3000,
                                               min_samples_split = 3,
                                               min_samples_leaf = 1, 
                                               max_leaf_nodes = 1000)
#random 3:
clf_RFR_set3 = ensemble.RandomForestClassifier(n_estimators  =  1000,
                                               min_samples_split = 1,
                                               min_samples_leaf = 1, 
                                               max_leaf_nodes = 500)
#random 4:
clf_RFR_set4 = ensemble.RandomForestClassifier(n_estimators  =  1000,
                                               min_samples_split = 3,
                                               min_samples_leaf = 1, 
                                               max_leaf_nodes = 500)
#random 5:
clf_RFR_set5 = ensemble.RandomForestClassifier(n_estimators  =  1000,
                                               min_samples_split = 1,
                                               min_samples_leaf = 3, 
                                               max_leaf_nodes = 500)

print ("Rando-1 Random Forest Regressor CV with scaling and normalization:")
ypred_1_RFR = line_CV_very_spicy(X, y, clf_RFR_set1, kf)
print ("Rando-2 Random Forest Regressor CV with scaling and normalization:")
ypred_2_RFR = line_CV_very_spicy(X, y, clf_RFR_set2, kf)
print ("Rando-3 Random Forest Regressor CV with scaling and normalization:")
ypred_3_RFR = line_CV_very_spicy(X, y, clf_RFR_set3, kf)
print ("Rando-4 Random Forest Regressor CV with scaling and normalization:")
ypred_4_RFR = line_CV_very_spicy(X, y, clf_RFR_set4, kf)
print ("Rando-5 Random Forest Regressor CV with scaling and normalization:")
ypred_5_RFR = line_CV_very_spicy(X, y, clf_RFR_set5, kf)

Rando-1 Random Forest Regressor CV with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 0.599
Root Mean Squared Logarithmic Error (higher; worse): 0.661
Root Mean Squared Logarithmic Error (higher; worse): 0.769
Root Mean Squared Logarithmic Error (higher; worse): 0.646
Root Mean Squared Logarithmic Error (higher; worse): 0.640
RMSLE Mean: 0.663


Rando-2 Random Forest Regressor CV with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 0.546
Root Mean Squared Logarithmic Error (higher; worse): 0.627
Root Mean Squared Logarithmic Error (higher; worse): 0.713
Root Mean Squared Logarithmic Error (higher; worse): 0.566
Root Mean Squared Logarithmic Error (higher; worse): 0.576
RMSLE Mean: 0.606


Rando-3 Random Forest Regressor CV with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 0.541
Root Mean Squared Logarithmic Error (higher; worse): 0.621
Root Mean Squared Logarithmic Error (higher; worse)

In [89]:
clf_RFR_set = ensemble.RandomForestClassifier(n_estimators  =  1000,
                                               max_leaf_nodes = 500)
print ("Rando-5 Random Forest Regressor CV with scaling and normalization:")
ypred_RFR = line_CV_very_spicy(X, y, clf_RFR_set, kf)

Rando-5 Random Forest Regressor CV with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 0.542
Root Mean Squared Logarithmic Error (higher; worse): 0.640
Root Mean Squared Logarithmic Error (higher; worse): 0.705
Root Mean Squared Logarithmic Error (higher; worse): 0.612
Root Mean Squared Logarithmic Error (higher; worse): 0.549
RMSLE Mean: 0.610




In [93]:
clf_RFR_set6 = RandomForestRegressor(n_estimators  =  1000,
                                     min_samples_split = 1,
                                     min_samples_leaf = 1, 
                                     max_leaf_nodes = 500)
print ("Rando-5 Random Forest Regressor CV with scaling and normalization:")
ypred_RFR6 = line_CV_very_spicy(XX, yy, clf_RFR_set6, kf)

Rando-5 Random Forest Regressor CV with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 0.392
Root Mean Squared Logarithmic Error (higher; worse): 0.404
Root Mean Squared Logarithmic Error (higher; worse): 0.447
Root Mean Squared Logarithmic Error (higher; worse): 0.421
Root Mean Squared Logarithmic Error (higher; worse): 0.445
RMSLE Mean: 0.422




In [99]:
clf_RFR_final = RandomForestRegressor(n_estimators  =  10000,
                             min_samples_split = 1,
                             min_samples_leaf = 1, 
                             max_leaf_nodes = 5000).fit(XX, yy)
Ypred1 = clf_RFR_final.predict(zz)


the output as the CSV file (that no floating number)

In [100]:
Ypred1.tofile('foo21.csv',sep=',',format='%.0f')


#  Support Vector Machine

**degree** : int, optional (default=3)
Degree of the polynomial kernel function (‘poly’). Ignored by all other kernels.

**gamma** : float, optional (default=’auto’)
Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’. If gamma is ‘auto’ then 1/n_features will be used instead.

**coef0**: float, optional (default=0.0)
Independent term in kernel function. It is only significant in ‘poly’ and ‘sigmoid’.

http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [ ]:
print("-- KNN Grid Parameter Search via 5-fold CV")

# set of parameters to test
C                  = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
kernel             = ['linear', 'poly', 'rbf', 'sigmoid']
degree             = [1, 2, 3, 4]
gamma              = [0, 1, 2, 3]
coef0              = [0, 1, 2, 3]
random_state       = [None, 5, 20, 100, 300]
param_grid_SVM    = dict(C=C,
                     kernel=kernel,
                     degree=degree,
                     gamma=gamma, 
                     coef0=coef0, random_state = random_state
                    )      

SVM_clf = svm.SVC()
SVM_ts_gs = run_gridsearch(X, y, SVM_clf, param_grid_SVM, cv=5)


1: 

`clf_SVM_set1 = svm.SVC(C  =  0.1, kernel = 'poly', degree = 5, coef0 = 4) `

2:

`clf_SVM_set1 = svm.SVC(C  =  30, kernel = 'rbf')`

3:

`clf_SVM_set4 = svm.SVC(C  =  50, kernel = 'linear')`



In [76]:
#random 1:
clf_SVM_set1 = svm.SVC(C  =  1, kernel = 'linear')
#random 2:
clf_SVM_set2 = svm.SVC(C  =  10, kernel = 'linear')
#random 3:
clf_SVM_set3 = svm.SVC(C  =  30, kernel = 'linear')
#random 4:
clf_SVM_set4 = svm.SVC(C  =  50, kernel = 'linear')
#random 5:
clf_SVM_set5 = svm.SVC(C  =  70, kernel = 'linear')


print ("Rando-1 SVM CV with scaling and normalization:")
ypred_1_SVM = line_CV_very_spicy(X, y, clf_SVM_set1, kf)
print ("Rando-2 SVM CV with scaling and normalization:")
ypred_2_SVM = line_CV_very_spicy(X, y, clf_SVM_set2, kf)
print ("Rando-3 SVM CV with scaling and normalization:")
ypred_3_SVM = line_CV_very_spicy(X, y, clf_SVM_set3, kf)
print ("Rando-4 SVM CV with scaling and normalization:")
ypred_4_SVM = line_CV_very_spicy(X, y, clf_SVM_set4, kf)
print ("Rando-5 SVM CV with scaling and normalization:")
ypred_5_SVM = line_CV_very_spicy(X, y, clf_SVM_set5, kf)

Rando-1 SVM CV with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 0.877
Root Mean Squared Logarithmic Error (higher; worse): 0.739
Root Mean Squared Logarithmic Error (higher; worse): 0.757
Root Mean Squared Logarithmic Error (higher; worse): 0.752
Root Mean Squared Logarithmic Error (higher; worse): 0.780
RMSLE Mean: 0.781


Rando-2 SVM CV with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 0.674
Root Mean Squared Logarithmic Error (higher; worse): 0.608
Root Mean Squared Logarithmic Error (higher; worse): 0.600
Root Mean Squared Logarithmic Error (higher; worse): 0.587
Root Mean Squared Logarithmic Error (higher; worse): 0.638
RMSLE Mean: 0.621


Rando-3 SVM CV with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 0.651
Root Mean Squared Logarithmic Error (higher; worse): 0.620
Root Mean Squared Logarithmic Error (higher; worse): 0.585
Root Mean Squared Logarithmic Error (higher; worse):

http://www.cs.toronto.edu/~duvenaud/cookbook/index.html

`clf_SVM_set1 = svm.SVC(C  =  0.1, kernel = 'poly', degree = 6, coef0 = 4, `



In [117]:
#random 1:
clf_SVM_set1 = svm.SVC(C  =  30, kernel = 'rbf')
#random 2:
clf_SVM_set2 = svm.SVC(C  =  40, kernel = 'rbf')
#random 3:
clf_SVM_set3 = svm.SVC(C  =  50, kernel = 'rbf')
#random 4:
clf_SVM_set4 = svm.SVC(C  =  20, kernel = 'rbf')
#random 5:
clf_SVM_set5 = svm.SVC(C  =  25, kernel = 'rbf')
#random 6:
clf_SVM_set6 = svm.SVC(C  =  35, kernel = 'rbf')


print ("Rando-1 SVM CV with scaling and normalization:")
ypred_11_SVM = line_CV_very_spicy(XX, yy, clf_SVM_set1, kf)
print ("Rando-2 SVM CV with scaling and normalization:")
ypred_22_SVM = line_CV_very_spicy(XX, yy, clf_SVM_set2, kf)
print ("Rando-3 SVM CV with scaling and normalization:")
ypred_33_SVM = line_CV_very_spicy(XX, yy, clf_SVM_set3, kf)
print ("Rando-4 SVM CV with scaling and normalization:")
ypred_44_SVM = line_CV_very_spicy(XX, yy, clf_SVM_set4, kf)
print ("Rando-5 SVM CV with scaling and normalization:")
ypred_55_SVM = line_CV_very_spicy(XX, yy, clf_SVM_set5, kf)
print ("Rando-6 SVM CV with scaling and normalization:")
ypred_66_SVM = line_CV_very_spicy(XX, yy, clf_SVM_set6, kf)

Rando-1 SVM CV with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 1.332
Root Mean Squared Logarithmic Error (higher; worse): 1.370
Root Mean Squared Logarithmic Error (higher; worse): 1.384
Root Mean Squared Logarithmic Error (higher; worse): 1.423
Root Mean Squared Logarithmic Error (higher; worse): 1.370
RMSLE Mean: 1.376


Rando-2 SVM CV with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 1.287
Root Mean Squared Logarithmic Error (higher; worse): 1.332
Root Mean Squared Logarithmic Error (higher; worse): 1.343
Root Mean Squared Logarithmic Error (higher; worse): 1.379
Root Mean Squared Logarithmic Error (higher; worse): 1.333
RMSLE Mean: 1.335


Rando-3 SVM CV with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): 1.067
Root Mean Squared Logarithmic Error (higher; worse): 1.095
Root Mean Squared Logarithmic Error (higher; worse): 1.094
Root Mean Squared Logarithmic Error (higher; worse):

/Users/Thwowu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:22: RuntimeWarning: invalid value encountered in log1p


Root Mean Squared Logarithmic Error (higher; worse): nan
Root Mean Squared Logarithmic Error (higher; worse): nan
Root Mean Squared Logarithmic Error (higher; worse): 0.902
Root Mean Squared Logarithmic Error (higher; worse): 0.921
Root Mean Squared Logarithmic Error (higher; worse): 0.974
RMSLE Mean: nan


Rando-5 SVM CV with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): nan
Root Mean Squared Logarithmic Error (higher; worse): nan
Root Mean Squared Logarithmic Error (higher; worse): nan
Root Mean Squared Logarithmic Error (higher; worse): nan
Root Mean Squared Logarithmic Error (higher; worse): nan
RMSLE Mean: nan


Rando-6 SVM CV with scaling and normalization:
Root Mean Squared Logarithmic Error (higher; worse): nan
Root Mean Squared Logarithmic Error (higher; worse): nan
Root Mean Squared Logarithmic Error (higher; worse): nan
Root Mean Squared Logarithmic Error (higher; worse): nan
Root Mean Squared Logarithmic Error (higher; worse): nan
RMSLE Mea

# Gradient Boosting 

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

**loss: **
‘deviance’, ‘exponential’

**max_depth:**
(default: 3)

maximum depth of the individual regression estimators. The maximum depth limits the number of nodes in the tree. **Tune this parameter** for best performance; the best value depends on the interaction of the input variables.




#### do we need to scale the data before doing GB? 

http://quant.stackexchange.com/questions/4434/gradient-tree-boosting-do-input-attributes-need-to-be-scaled

Overall, you should probably do both (with and without). 
And recall, there are choices for scaling: 

(a) you can remove skewness (fat-tails)

(b) mean-zero standardize

(c) normalize in range [0,1]

(d) determine direction cosines of the input features so feature values are within a unit circle.

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html


https://people.duke.edu/~rnau/compare.htm


**The mean absolute scaled error (MASE)** is another relative measure of error that is applicable only to time series data.  It is defined as the mean absolute error of the model divided by the mean absolute error of a naïve random-walk-without-drift model (i.e., the mean absolute value of the first difference of the series).  Thus, it measures the relative reduction in error compared to a naive model. **Ideally its value will be significantly less than 1. ** This statistic, which was proposed by Rob Hyndman in 2006, is very good to look at when fitting regression models to nonseasonal time series data.  It is possible for a time series regression model to have an impressive R-squared and yet be inferior to a naïve model, as was demonstrated in the what’s-a-good-value-for-R-squared notes.  If the series has a strong seasonal pattern, the corresponding statistic to look at would be the mean absolute error divided by the mean absolute value of the seasonal difference (i.e., the mean absolute error of a naïve seasonal model that predicts that the value in a given period will equal the value observed one season ago).


In [93]:
#random 1:
clf_GB_set1 = ensemble.GradientBoostingRegressor(n_estimators = 800, 
                                                 max_depth = 7, 
                                                 learning_rate = 0.05)
#random 2:
clf_GB_set2 = ensemble.GradientBoostingRegressor(n_estimators = 1000, 
                                                 max_depth = 7, 
                                                 learning_rate = 0.05)
#random 3:
clf_GB_set3 = ensemble.GradientBoostingRegressor(n_estimators = 1200, 
                                                 max_depth = 7, 
                                                 learning_rate = 0.1)
#random 4:
clf_GB_set4 = ensemble.GradientBoostingRegressor(n_estimators = 1500, 
                                                 max_depth = 7, 
                                                 learning_rate = 0.2)
#random 5:
clf_GB_set5 = ensemble.GradientBoostingRegressor(n_estimators = 1800, 
                                                 max_depth = 7, 
                                                 learning_rate = 0.05)
#random 6:
clf_GB_set6 = ensemble.GradientBoostingRegressor(n_estimators = 2000, 
                                                 max_depth = 7, 
                                                 loss = 'huber', 
                                                 learning_rate = 0.03)


print ("Rando-1 Gradient Boosting")
ypred_11_GB = CV_with_MSE(XX, yy, clf_GB_set1, kf)
print ("Rando-2 Gradient Boosting")
ypred_22_GB = CV_with_MSE(XX, yy, clf_GB_set2, kf)

print ("Rando-3 Gradient Boosting")
ypred_33_GB = CV_with_MSE(XX, yy, clf_GB_set3, kf)
print ("Rando-4 Gradient Boosting")
ypred_44_GB = CV_with_MSE(XX, yy, clf_GB_set4, kf)
print ("Rando-5 Gradient Boosting")
ypred_55_GB = CV_with_MSE(XX, yy, clf_GB_set5, kf)
print ("Rando-6 Gradient Boosting")
ypred_66_GB = CV_with_MSE(XX, yy, clf_GB_set6, kf)


Rando-1 Gradient Boosting
Root Mean Squared Error: 39.013, R Square mean: 0.955
Mean absolute percentage error: 31.970, Mean absolute scaled error: 1.503
Root Mean Squared Error: 39.896, R Square mean: 0.951
Mean absolute percentage error: 32.135, Mean absolute scaled error: 1.509
Root Mean Squared Error: 37.768, R Square mean: 0.956
Mean absolute percentage error: 27.201, Mean absolute scaled error: 1.426
Root Mean Squared Error: 41.238, R Square mean: 0.948
Mean absolute percentage error: 30.297, Mean absolute scaled error: 1.530
Root Mean Squared Error: 41.750, R Square mean: 0.947
Mean absolute percentage error: 29.241, Mean absolute scaled error: 1.539
RMSE Mean: 39.933, R Square mean: 0.951
MAPE: 30.169, MASE: 1.501


Rando-2 Gradient Boosting
Root Mean Squared Error: 39.156, R Square mean: 0.954
Mean absolute percentage error: 31.936, Mean absolute scaled error: 1.506
Root Mean Squared Error: 39.803, R Square mean: 0.951
Mean absolute percentage error: 32.597, Mean absolute scal

In [ ]:
'accuracy', 'adjusted_rand_score', 'average_precision', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'log_loss', 'mean_absolute_error', 'mean_squared_error', 'median_absolute_error', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc'

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.grid_search import GridSearchCV as GS
from sklearn import ensemble

"""num_folds = 10
num_instances = len(X)
seed = 7
num_trees = 100
max_features = 3
kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
results = cross_validation.cross_val_score(model, X, y, cv=kfold)
"""

### Extreme Boosting

http://xgboost.readthedocs.io/en/latest/python/python_intro.html


http://machinelearningmastery.com/tune-learning-rate-for-gradient-boosting-with-xgboost-in-python/